In [ ]:
!pip3 install transformers

In [7]:
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
bert_model = BertForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased').eval()
bert_tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
top_k = 10

def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'
    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

def get_all_predictions(text_sentence, top_clean):
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    #print('Input: ', input_ids)
    #print('Mask: ', mask_idx)
  
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
    return bert

In [22]:
def get_predictions(input_text, best_n_predictions=1):
    try:
        input_text += ' <mask>'
        res = get_all_predictions(input_text, best_n_predictions)
        return res

    except Exception as error:
        err = str(error)
        return err

In [ ]:
input_text = "Dinheiro não traz"

print(get_predictions(input_text,best_n_predictions=1))